In [69]:
import nltk
import numpy as np
import pandas as pd
import os
import itertools
import collections
from datetime import datetime
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter
from nltk import FreqDist
from nltk import bigrams
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import pickle
import re
import pyLDAvis
import pyLDAvis.gensim_models

In [133]:
Spadek_2021 = pd.read_excel(r'C:\Users\Bajdz\Desktop\Spadek_2021.xlsx')
Wzrost_2021 = pd.read_excel(r'C:\Users\Bajdz\Desktop\Wzrost_2020_2021.xlsx')
Wzrost_2017 = pd.read_excel(r'C:\Users\Bajdz\Desktop\Wzrost_2017.xlsx')
Spadek_2017 = pd.read_excel(r'C:\Users\Bajdz\Desktop\Spadek_2017.xlsx')

Wzrost_2021['Text'].dropna(inplace=True)
Spadek_2021['Text'].dropna(inplace=True)
Spadek_2017['Text'].dropna(inplace=True)
Wzrost_2017['Text'].dropna(inplace=True)

Spadek_2021["Date"] = pd.to_datetime(Spadek_2021["Date"])
Wzrost_2021["Date"] = pd.to_datetime(Wzrost_2021["Date"])
Wzrost_2017["Date"] = pd.to_datetime(Wzrost_2017["Date"])
Spadek_2017["Date"] = pd.to_datetime(Spadek_2017["Date"])

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))

text_Spadek_2021 = Spadek_2021['Text'].copy()
text_Wzrost_2021 = Wzrost_2021['Text'].copy()
text_Wzrost_2017 = Wzrost_2017['Text'].copy()
text_Spadek_2017 = Spadek_2017['Text'].copy()

In [134]:
#Liczba słow na komentarz
x = 0
for i in range(len(text_Spadek_2021)):
    x+=len(text_Spadek_2021[i])
print(x/len(text_Spadek_2021))

x = 0
for i in range(len(text_Wzrost_2021)):
    x+=len(text_Wzrost_2021[i])
print(x/len(text_Wzrost_2021))

x = 0
for i in range(len(text_Wzrost_2017)):
    x+=len(text_Wzrost_2017[i])
print(x/len(text_Wzrost_2017))

x = 0
for i in range(len(text_Spadek_2017)):
    x+=len(text_Spadek_2017[i])
x/len(text_Spadek_2017)

6.230692076228686
6.240285674012123
6.473412629001224


7.041122091169908

In [135]:

text_Spadek_2021 = text_Spadek_2021.str.strip()
text_Wzrost_2021 = text_Wzrost_2021.str.strip()
text_Wzrost_2017 = text_Wzrost_2017.str.strip()
text_Spadek_2017 = text_Spadek_2017.str.strip()

text_Spadek_2021 = text_Spadek_2021.str.lower() 
text_Wzrost_2021 = text_Wzrost_2021.str.lower() 
text_Wzrost_2017 = text_Wzrost_2017.str.lower() 
text_Spadek_2017 = text_Spadek_2017.str.lower() 

text_Spadek_2021 = text_Spadek_2021.tolist()
text_Wzrost_2021 = text_Wzrost_2021.tolist()
text_Wzrost_2017 = text_Wzrost_2017.tolist()
text_Spadek_2017 = text_Spadek_2017.tolist()

for i in range(len(text_Spadek_2017)):
    text_Spadek_2017[i] = tokenizer.tokenize(text_Spadek_2017[i])
for i in range(len(text_Spadek_2021)):
    text_Spadek_2021[i] = tokenizer.tokenize(text_Spadek_2021[i])
for i in range(len(text_Wzrost_2017)):
    text_Wzrost_2017[i] = tokenizer.tokenize(text_Wzrost_2017[i])
for i in range(len(text_Wzrost_2021)):
    text_Wzrost_2021[i] = tokenizer.tokenize(text_Wzrost_2021[i])
    
#Delete common words 
text_Spadek_2021 = [[token for token in teksty if token not in stop_words] for teksty in text_Spadek_2021] 
text_Wzrost_2021 = [[token for token in teksty if token not in stop_words] for teksty in text_Wzrost_2021] 
text_Wzrost_2017 = [[token for token in teksty if token not in stop_words] for teksty in text_Wzrost_2017] 
text_Spadek_2017 = [[token for token in teksty if token not in stop_words] for teksty in text_Spadek_2017] 



In [136]:
#Delete short words
text_Spadek_2021 = [[token for token in teksty if len(token) > 2] for teksty in text_Spadek_2021] 
text_Wzrost_2021 = [[token for token in teksty if len(token) > 2] for teksty in text_Wzrost_2021] 
text_Wzrost_2017 = [[token for token in teksty if len(token) > 2] for teksty in text_Wzrost_2017] 
text_Spadek_2017 = [[token for token in teksty if len(token) > 2] for teksty in text_Spadek_2017] 

#Delete numeric
text_Spadek_2021 = [[token for token in com if not token.isnumeric()] for com in text_Spadek_2021]
text_Wzrost_2021 = [[token for token in com if not token.isnumeric()] for com in text_Wzrost_2021]
text_Wzrost_2017 = [[token for token in com if not token.isnumeric()] for com in text_Wzrost_2017]
text_Spadek_2017 = [[token for token in com if not token.isnumeric()] for com in text_Spadek_2017]

#Lematization of words
lemmatizer = WordNetLemmatizer()
text_Spadek_2021 = [[lemmatizer.lemmatize(token) for token in teksty] for teksty in text_Spadek_2021]
text_Wzrost_2021 = [[lemmatizer.lemmatize(token) for token in teksty] for teksty in text_Wzrost_2021]
text_Wzrost_2017 = [[lemmatizer.lemmatize(token) for token in teksty] for teksty in text_Wzrost_2017]
text_Spadek_2017 = [[lemmatizer.lemmatize(token) for token in teksty] for teksty in text_Spadek_2017]

#create bigrams and trigrams
bigram_Spadek_2017 = Phrases(text_Spadek_2017, min_count = 15)
for i in range(len(text_Spadek_2017)):
    for token in bigram_Spadek_2017[text_Spadek_2017[i]]:
        if '_' in token:
            text_Spadek_2017[i].append(token)
            
trigram_Spadek_2017 = Phrases(bigram_Spadek_2017[text_Spadek_2017], min_count = 3) 
for i in range(len(text_Spadek_2017)):
    for token in trigram_Spadek_2017[bigram_Spadek_2017[text_Spadek_2017[i]]]:
        if '_' in token:
            text_Spadek_2017[i].append(token)
            
bigram_Wzrost_2021 = Phrases(text_Wzrost_2021, min_count=70)
for i in range(len(text_Wzrost_2021)):
    for token in bigram_Wzrost_2021[text_Wzrost_2021[i]]:
        if '_' in token:
            text_Wzrost_2021[i].append(token)
            
trigram_Wzrost_2021 = Phrases(bigram_Wzrost_2021[text_Wzrost_2021], min_count = 15) 
for i in range(len(text_Wzrost_2021)):
    for token in trigram_Wzrost_2021[bigram_Wzrost_2021[text_Wzrost_2021[i]]]:
        if '_' in token:
            text_Wzrost_2021[i].append(token)
            
bigram_Wzrost_2017 = Phrases(text_Wzrost_2017, min_count=45)
for i in range(len(text_Wzrost_2017)):
    for token in bigram_Wzrost_2017[text_Wzrost_2017[i]]:
        if '_' in token:
            text_Wzrost_2017[i].append(token)
            
trigram_Wzrost_2017 = Phrases(bigram_Wzrost_2017[text_Wzrost_2017], min_count = 8) 
for i in range(len(text_Wzrost_2017)):
    for token in trigram_Wzrost_2017[bigram_Spadek_2017[text_Wzrost_2017[i]]]:
        if '_' in token:
            text_Wzrost_2017[i].append(token)
            
bigram_Spadek_2021 = Phrases(text_Spadek_2021, min_count=22)
for i in range(len(text_Spadek_2021)):
    for token in bigram_Spadek_2021[text_Spadek_2021[i]]:
        if '_' in token:
            text_Spadek_2021[i].append(token)
            
trigram_Spadek_2021 = Phrases(bigram_Spadek_2021[text_Spadek_2021], min_count = 3) 
for i in range(len(text_Spadek_2021)):
    for token in trigram_Spadek_2021[bigram_Spadek_2021[text_Spadek_2021[i]]]:
        if '_' in token:
            text_Spadek_2021[i].append(token)
     
#Delete stopwords
to_delete = ["soon","day","lol","next","good","shiba_inu","year","make","look_like","let_see","next_week","take","look","think","http","still","look_like_look_like","next","good_luck_good_luck","day","stock_market","next_hour","http_invst_http_invst","http_www_http_www","today","chart","make","gray_box","gray_box_gray_box","support","right","gray_box_gray_box_gray_box","look_like_look_like_look_like","level","last","gray","http_invst","https_www","btc","bitcoin","looks_like", "http","day","said","â_â","https_invst","real","tomorrow","post","ago","doe","wa","ha","even","term","bounce","http","guy","coin","christmas","well","u","come","every","wait","lot","exchange","close","never","watch","made","go","volume","start","first","day","already","may","hour","week","hi_guy","however","iâ","ł","cant","http_www","like","add","case","donâ","thats","second","hey","due", "đź","đź_łđź","łđź","per","etc","http_twitter", "people","actually","seen","follow", "get", "see", "back","thank","man", "within","ur","opinion","thanks","crypto_currency","place","feel","two","le","near","likely","went","like","see","go","going","people","000","think","guy","look","http","know","want","crypto","sl","either","old","touch","took","would","way","http_www","could","news","u","dont","many","http_invst","another","anyone","00","hr","ťâ","still","big","com","let","say","coming","much","need","thing","move","point","around","please","everyone","value","since","might","đźš","itâ","đź_đź","đź","taking","ok","hi","ly","else","www","seems","gonna","world","im","also","đźš_đźš","đźš","tp","use","yet","thought","probably","till","le","try","better","forum","line","u","v","comment","little","ever","tell","oh","yes","away","become","want","though","retest","anybody","folk","eye","sec","ask","suggest","saw","neo","html","guy","letâ","one","market"]
stop_words.update(to_delete) 

text_Spadek_2021 = [[token for token in teksty if token not in stop_words] for teksty in text_Spadek_2021] 
text_Wzrost_2021 = [[token for token in teksty if token not in stop_words] for teksty in text_Wzrost_2021] 
text_Wzrost_2017 = [[token for token in teksty if token not in stop_words] for teksty in text_Wzrost_2017] 
text_Spadek_2017 = [[token for token in teksty if token not in stop_words] for teksty in text_Spadek_2017] 

#Create Dictionarises
dictionary_Spadek_2021 = Dictionary(text_Spadek_2021)
dictionary_Spadek_2021.filter_extremes(no_below=15, no_above=0.3)

dictionary_Wzrost_2021 = Dictionary(text_Wzrost_2021)
dictionary_Wzrost_2021.filter_extremes(no_below=100, no_above=0.3)

dictionary_Wzrost_2017 = Dictionary(text_Wzrost_2017)
dictionary_Wzrost_2017.filter_extremes(no_below=65, no_above=0.3)

dictionary_Spadek_2017 = Dictionary(text_Spadek_2017)
dictionary_Spadek_2017.filter_extremes(no_below=28, no_above=0.3)


#liczenie slow i bigramow
Spadek_2021_to_count = [item for t in text_Spadek_2021 for item in t]
counted_Spadek_2021 = Counter(Spadek_2021_to_count)

Wzrost_2021_to_count = [item for t in text_Wzrost_2021 for item in t]
counted_Wzrost_2021 = Counter(Wzrost_2021_to_count)

Wzrost_2017_to_count = [item for t in text_Wzrost_2017 for item in t]
counted_Wzrost_2017 = Counter(Wzrost_2017_to_count)

Spadek_2017_to_count = [item for t in text_Spadek_2017 for item in t]
counted_Spadek_2017 = Counter(Spadek_2017_to_count)




In [137]:
from gensim.models import TfidfModel

corpus_Spadek_2021 = [dictionary_Spadek_2021.doc2bow(teksty) for teksty in text_Spadek_2021] 
model_Spadek_2021 = TfidfModel(corpus_Spadek_2021)

[[(dictionary_Spadek_2021[i], freq) for i, freq in teksty] for teksty in corpus_Spadek_2021] 
print('unikalne tokeny:', len(dictionary_Spadek_2021))
print('komentarze:', len(corpus_Spadek_2021))

corpus_Wzrost_2021 = [dictionary_Wzrost_2021.doc2bow(teksty) for teksty in text_Wzrost_2021] 
model_Wzrost_2021 = TfidfModel(corpus_Wzrost_2021)
[[(dictionary_Wzrost_2021[i], freq) for i, freq in teksty] for teksty in corpus_Wzrost_2021] 
print('unikalne tokeny:', len(dictionary_Wzrost_2021))
print('komentarze:', len(corpus_Wzrost_2021))

corpus_Spadek_2017 = [dictionary_Spadek_2017.doc2bow(teksty) for teksty in text_Spadek_2017] 
model_Spadek_2017  = TfidfModel(corpus_Spadek_2017)
[[(dictionary_Spadek_2017[i], freq) for i, freq in teksty] for teksty in corpus_Spadek_2017] 
print('unikalne tokeny:', len(dictionary_Spadek_2017))
print('komentarze:', len(corpus_Spadek_2017))

corpus_Wzrost_2017 = [dictionary_Wzrost_2017.doc2bow(teksty) for teksty in text_Wzrost_2017] 
model_Wzrost_2017 = TfidfModel(corpus_Wzrost_2017)
[[(dictionary_Wzrost_2017[i], freq) for i, freq in teksty] for teksty in corpus_Wzrost_2017] 
print('unikalne tokeny:', len(dictionary_Wzrost_2017))
print('komentarze:', len(corpus_Wzrost_2017))


unikalne tokeny: 112
komentarze: 1994
unikalne tokeny: 193
komentarze: 21283
unikalne tokeny: 105
komentarze: 3137
unikalne tokeny: 176
komentarze: 11434


In [138]:
#Trenowanie modelu
num_topics = 4
passes = 30
iter = 400
eval = None

In [139]:
def compute_coherence_values(dictionary, id2word, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iter,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval
    )
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [140]:
import matplotlib.pyplot as plt
#Spadki 2021
chunksize = 100
vector = model_Spadek_2021[corpus_Spadek_2021]
temp = dictionary_Spadek_2021[0]  # temp, zeby zaladowac slownik
id2word = dictionary_Spadek_2021.id2token

model_Spadek_2021 = LdaModel(random_state =2021,
    corpus=corpus_Spadek_2021,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iter,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval
)

top_topics_Spadek_2021 = model_Spadek_2021.top_topics(corpus_Spadek_2021) #, num_words=20) # pierwsza wartosc - coherence score, spojnosc slownictwa w temacie.
avg_topic_coherence = sum([t[1] for t in top_topics_Spadek_2021]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)
print('\nPerplexity: ', model_Spadek_2021.log_perplexity(corpus_Spadek_2021))
doc_lda_Spadek_2021 = model_Spadek_2021[corpus_Spadek_2021]
#Coherence Score
coherence_model_lda = CoherenceModel(model=model_Spadek_2021, texts=text_Spadek_2021, dictionary=dictionary_Spadek_2021, coherence='c_v')
coherence_lda_Spadek_2021 = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda_Spadek_2021)
#from pprint import pprint
#pprint(top_topics_Spadek_2021)



Average topic coherence: -12.7959.

Perplexity:  -4.559259837386756
Coherence Score:  0.4511759321445705


In [141]:
#Spadki 2017
chunksize = 100
temp = dictionary_Spadek_2017[0]  # temp, zeby zaladowac slownik
id2word = dictionary_Spadek_2017.id2token
vector = model_Spadek_2017[corpus_Spadek_2017]
model_Spadek_2017 = LdaModel(random_state =2021,
    corpus=corpus_Spadek_2017,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iter,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval
)

top_topics_Spadek_2017 = model_Spadek_2017.top_topics(corpus_Spadek_2017) #, num_words=20) # pierwsza wartosc - coherence score, spojnosc slownictwa w temacie.
avg_topic_coherence = sum([t[1] for t in top_topics_Spadek_2017]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)
doc_lda_Spadek_2017 = model_Spadek_2017[corpus_Spadek_2017]
print('\nPerplexity: ', model_Spadek_2017.log_perplexity(corpus_Spadek_2017))
#Coherence Score
coherence_model_lda = CoherenceModel(model=model_Spadek_2017, texts=text_Spadek_2017, dictionary=dictionary_Spadek_2017, coherence='c_v')
coherence_lda_Spadek_2017 = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda_Spadek_2017)


Average topic coherence: -8.9840.

Perplexity:  -4.548203366894605
Coherence Score:  0.32277881787629065


In [142]:
#Wzrosty 2017
chunksize = 100
temp = dictionary_Wzrost_2017[0]  # temp, zeby zaladowac slownik
id2word = dictionary_Wzrost_2017.id2token
vector = model_Wzrost_2017[corpus_Wzrost_2017]
model_Wzrost_2017 = LdaModel(random_state =2021,
    corpus=corpus_Wzrost_2017,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iter,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval
)

top_topics_Wzrost_2017 = model_Wzrost_2017.top_topics(corpus_Wzrost_2017) #, num_words=20) # pierwsza wartosc - coherence score, spojnosc slownictwa w temacie.
avg_topic_coherence = sum([t[1] for t in top_topics_Wzrost_2017]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)
doc_lda_Wzrost_2017 = model_Wzrost_2017[corpus_Wzrost_2017]
print('\nPerplexity: ', model_Wzrost_2017.log_perplexity(corpus_Wzrost_2017))

#Coherence Score
coherence_model_lda = CoherenceModel(model=model_Wzrost_2017, texts=text_Wzrost_2017, dictionary=dictionary_Wzrost_2017, coherence='c_v')
coherence_lda_Wzrost_2017 = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda_Wzrost_2017)


Average topic coherence: -5.6432.

Perplexity:  -5.033457469028007
Coherence Score:  0.23450284873268967


In [143]:
#Wzrosty 2021
chunksize = 100
temp = dictionary_Wzrost_2021[0]  # temp, zeby zaladowac slownik
id2word = dictionary_Wzrost_2021.id2token
vector = model_Wzrost_2021[corpus_Wzrost_2021]
model_Wzrost_2021 = LdaModel(random_state =2021,
    corpus=corpus_Wzrost_2021,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iter,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval
)

top_topics_Wzrost_2021 = model_Wzrost_2021.top_topics(corpus_Wzrost_2021) #, num_words=20) # pierwsza wartosc - coherence score, spojnosc slownictwa w temacie.
avg_topic_coherence = sum([t[1] for t in top_topics_Wzrost_2021]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)
doc_lda_Wzrost_2021 = model_Wzrost_2017[corpus_Wzrost_2021]
print('\nPerplexity: ', model_Wzrost_2021.log_perplexity(corpus_Wzrost_2021))

#Coherence Score
coherence_model_lda = CoherenceModel(model=model_Wzrost_2021, texts=text_Wzrost_2021, dictionary=dictionary_Wzrost_2021, coherence='c_v')
coherence_lda_Wzrost_2021 = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda_Wzrost_2021)



Average topic coherence: -4.8194.

Perplexity:  -5.030280602886793
Coherence Score:  0.238012372353069


In [144]:
#Wizualizacja
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(model_Spadek_2021, corpus_Spadek_2021, dictionary_Spadek_2021)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.367744  0.120761       1        1  31.369114
3     -0.263164  0.270477       2        1  29.506946
1     -0.074152 -0.339993       3        1  21.719403
2     -0.030427 -0.051246       4        1  17.404537, topic_info=          Term        Freq       Total Category  logprob  loglift
6          buy  269.000000  269.000000  Default  30.0000  30.0000
42        sell  156.000000  156.000000  Default  29.0000  29.0000
85        elon  195.000000  195.000000  Default  28.0000  28.0000
86   elon_musk  185.000000  185.000000  Default  27.0000  27.0000
11        time   99.000000   99.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
38        hard    0.267930   24.801083   Topic4  -7.8591  -2.7795
72    ethereum    0.267640   45.665858   Topic4  -7.8602  -3.3910
26      always    0.266072   38.208541   Topic4  -7.8661  -3.2186
13      dollar    0.263435   29.370644   Topic4  -7.8761  -2.9655
90  correction    0.264016   35.261172   Topic4  -7.8738  -3.1461

[171 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
35        1  0.981047     100k
60        2  0.951439      10k
0         1  0.956453      30k
73        1  0.979835      40k
94        3  0.973545      42k
...     ...       ...      ...
62        2  0.970422  waiting
52        4  0.976989    whale
81        4  0.978374    worth
89        3  0.957813      wow
101       2  0.969805     zero

[112 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3])

In [145]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(model_Spadek_2017, corpus_Spadek_2017, dictionary_Spadek_2017)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.404763  0.019852       1        1  30.552415
1     -0.164135  0.341349       2        1  25.505435
3     -0.137688 -0.311095       3        1  22.754199
2     -0.102941 -0.050106       4        1  21.187951, topic_info=      Term        Freq       Total Category  logprob  loglift
11     buy  415.000000  415.000000  Default  30.0000  30.0000
4    price  199.000000  199.000000  Default  29.0000  29.0000
22   short  195.000000  195.000000  Default  28.0000  28.0000
23    sell  231.000000  231.000000  Default  27.0000  27.0000
5     time  222.000000  222.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
87  trader    0.296429   89.776015   Topic4  -8.4499  -4.1615
27    stay    0.289948   44.393436   Topic4  -8.4720  -3.4794
56   trade    0.294651  106.405376   Topic4  -8.4559  -4.3375
20    hold    0.293483   92.522786   Topic4  -8.4599  -4.2017
98     10k    0.293000  106.765603   Topic4  -8.4615  -4.3465

[169 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
98        3  0.992829     10k
15        1  0.982054     15k
24        1  0.959378     20k
81        4  0.939102  almost
16        4  0.992274     alt
...     ...       ...     ...
28        3  0.985751   trend
52        2  0.994569     usd
92        4  0.979002    wall
61        3  0.972486     wow
89        4  0.964941     xrp

[105 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3])

In [146]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(model_Wzrost_2017, corpus_Wzrost_2017, dictionary_Wzrost_2017)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.407061  0.079627       1        1  32.676732
0     -0.218394  0.341915       2        1  25.153441
2     -0.080799 -0.118643       3        1  21.689293
1     -0.107867 -0.302899       4        1  20.480533, topic_info=       Term         Freq        Total Category  logprob  loglift
81      bch  1843.000000  1843.000000  Default  30.0000  30.0000
18    price   856.000000   856.000000  Default  29.0000  29.0000
6       buy   990.000000   990.000000  Default  28.0000  28.0000
59    short   776.000000   776.000000  Default  27.0000  27.0000
20     sell   494.000000   494.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
52     done    95.133369    96.011485   Topic4  -4.0780   1.5765
153  minute    90.446686    91.311733   Topic4  -4.1285   1.5762
100   daily    86.651766    87.499095   Topic4  -4.1714   1.5760
104    area    87.541428    88.404825   Topic4  -4.1611   1.5759
114   later    84.316086    85.172837   Topic4  -4.1987   1.5756

[151 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
170       4  0.996488        10k
29        3  0.990330     always
104       4  0.995421       area
124       2  0.995649        ath
41        2  0.995341        bad
...     ...       ...        ...
40        3  0.988651    without
43        2  0.993525       work
160       1  0.992768        wow
109       3  0.993580        xrp
134       2  0.994644  yesterday

[121 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])

In [147]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(model_Wzrost_2021, corpus_Wzrost_2021, dictionary_Wzrost_2021)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.407212 -0.010906       1        1  29.334232
2     -0.191228 -0.339110       2        1  27.898522
3     -0.174394  0.345239       3        1  26.055146
1     -0.041590  0.004777       4        1  16.712101, topic_info=       Term         Freq        Total Category  logprob  loglift
6       buy  2522.000000  2522.000000  Default  30.0000  30.0000
23    price  1496.000000  1496.000000  Default  29.0000  29.0000
4      sell  1362.000000  1362.000000  Default  28.0000  28.0000
17    short  1216.000000  1216.000000  Default  27.0000  27.0000
7      time  1091.000000  1091.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
171     10k   131.387139   132.137092   Topic4  -4.1093   1.7833
147  candle   124.085613   124.807980   Topic4  -4.1665   1.7832
183    gain   127.629282   128.382559   Topic4  -4.1383   1.7832
132    fund   123.830272   124.582512   Topic4  -4.1685   1.7830
9      safe   119.085349   119.816246   Topic4  -4.2076   1.7829

[153 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
139       4  0.998492     100k
171       4  0.991395      10k
182       3  0.998031      20k
190       1  0.997739      25k
188       1  0.998473      27k
...     ...       ...      ...
88        1  0.998195  waiting
140       1  0.997220    whale
52        2  0.995451    worth
111       1  0.994213    wrong
104       2  0.997216      xrp

[123 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2])